In [1]:
%cd mmdetection

/opt/ml/DACON-objectdetection/mmdetection


In [2]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmcv.parallel import MMDataParallel
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [3]:
classes = ('chevrolet_malibu_sedan_2012_2016', 'chevrolet_malibu_sedan_2017_2019', 'chevrolet_spark_hatchback_2016_2021', 
           'chevrolet_trailblazer_suv_2021_', 'chevrolet_trax_suv_2017_2019','genesis_g80_sedan_2016_2020','genesis_g80_sedan_2021_', 
           'genesis_gv80_suv_2020_', 'hyundai_avante_sedan_2011_2015', 'hyundai_avante_sedan_2020_', 'hyundai_grandeur_sedan_2011_2016', 
           'hyundai_grandstarex_van_2018_2020', 'hyundai_ioniq_hatchback_2016_2019', 'hyundai_sonata_sedan_2004_2009', 
           'hyundai_sonata_sedan_2010_2014', 'hyundai_sonata_sedan_2019_2020', 'kia_carnival_van_2015_2020', 'kia_carnival_van_2021_',
           'kia_k5_sedan_2010_2015', 'kia_k5_sedan_2020_', 'kia_k7_sedan_2016_2020', 'kia_mohave_suv_2020_', 'kia_morning_hatchback_2004_2010',
           'kia_morning_hatchback_2011_2016', 'kia_ray_hatchback_2012_2017', 'kia_sorrento_suv_2015_2019', 'kia_sorrento_suv_2020_', 'kia_soul_suv_2014_2018',
           'kia_sportage_suv_2016_2020','kia_stonic_suv_2017_2019','renault_sm3_sedan_2015_2018','renault_xm3_suv_2020_','ssangyong_korando_suv_2019_2020',
           'ssangyong_tivoli_suv_2016_2020'),

# config file 들고오기
cfg = Config.fromfile('./mmdetection/configs/_HJ_/cascade-mask-rcnn_convnext-t.py')

root='./data/'
working_dir = './mmdetection/work_dirs/cascade-mask-rcnn_convnext-t'

epoch = 'best_bbox_mAP_epoch_20'

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

In [4]:
# dataset config 수정
cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'annotations/test.json'
cfg.data.test.test_mode = True

In [5]:
# cfg.data.test.pipeline
# cfg.data.train.pipeline

[{'type': 'LoadImageFromFile'},
 {'type': 'MultiScaleFlipAug',
  'img_scale': [(800, 1333), (1600, 2666)],
  'flip': False,
  'transforms': [{'type': 'Resize',
    'multiscale_mode': 'range',
    'keep_ratio': True},
   {'type': 'Normalize',
    'mean': [123.675, 116.28, 103.53],
    'std': [58.395, 57.12, 57.375],
    'to_rgb': True},
   {'type': 'Pad', 'size_divisor': 32},
   {'type': 'ImageToTensor', 'keys': ['img']},
   {'type': 'Collect', 'keys': ['img']}]}]

In [ ]:
# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

In [ ]:
# checkpoint path
checkpoint_path = os.path.join(working_dir, f'{epoch}.pth')

model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

In [ ]:
output = single_gpu_test(model, data_loader) # output 계산

In [ ]:
def coco_to_labelme(line, image_width, image_height, txt_file_name):    
    file_name = txt_file_name.split("/")[-1].replace(".txt", ".png")
    class_id, x, y, width, height, confidence = [float(temp) for temp in line.split()]
    
    x_min = int((x - width / 2) * image_width)
    x_max = int((x + width / 2) * image_width)
    y_min = int((y - height / 2) * image_height)
    y_max = int((y + height / 2) * image_height)
    
    return file_name, int(class_id), confidence, x_min, y_min, x_max, y_min, x_max, y_max, x_min, y_max

In [ ]:
# submission 양식에 맞게 output 후처리
results = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

class_num = 34
for i, out in enumerate(output):
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            # o[0], o[1], o[2], o[3] : x_min, y_min, x_max, y_max
            # file_name,class_id,confidence,point1_x,point1_y,point2_x,point2_y,point3_x,point3_y,point4_x,point4_y
            results.append([image_info['file_name'][-13:], j, o[4], o[0], o[1], o[2], o[1], o[2], o[3], o[0], o[3]])

In [ ]:
submission = pd.DataFrame(data=results, columns=["file_name", "class_id", "confidence", "point1_x", "point1_y", "point2_x", "point2_y", "point3_x", "point3_y", "point4_x", "point4_y"])
submission.to_csv(os.path.join(working_dir, f'submission_{epoch}.csv'), index=None)

In [ ]:
submission.head()